In [1]:
import json
import os
import sys  
sys.path.insert(0, '..')
import numpy as np
import platform
import mobilenetyolo as model
import utillity as utill
import models
import yolo_utils
import cv2

from tqdm import tqdm
from tensorflow.keras import layers, models as mdl, backend as k
from mobilenetyolo import MNetYolo
from matplotlib import image, pyplot as plt, patches as patches


2022-01-07 15:15:19.611105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-07 15:15:19.611144: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
weight_path = os.path.join('..', 'assets', 'mobilenet-yolo-light.01-1222.80.h5')
val_notation = os.path.join('..', 'assets', 'validation.txt')
train_notation = os.path.join('..', 'assets', 'training.txt')
predict_path = os.path.join('..', 'assets', 'mnet_yolo', 'predict')
gt_path = os.path.join('..', 'assets', 'mnet_yolo', 'ground_truth')
output_ap = os.path.join('..', 'assets', 'mnet_yolo', 'result')
temp_json = os.path.join('..', 'assets', 'mnet_yolo','temp_json') 
class_name = os.path.join('..', 'assets', 'class_name.txt')
image_path = os.path.join('/home', 'hadioz', 'Documents', 'skripsi', 'dataset', 'WIDER_val', 'images')

In [3]:
config = { 
  'image_size' : (416, 416, 3),
  'anchors' : [ 3, 5, 7, 12, 12, 22, 19, 33, 26, 51, 52, 47, 39, 76, 54, 111, 98, 94, 79, 163, 157, 153, 113, 236, 224, 222, 183, 338 ],
  'strides' : [ 16, 32 ],
  'xyscale':[ 0.1, 0.05 ],
  'detector_count' : 2,
  'anchor_size_perdetector': 7,

  # Training
  'iou_loss_thresh': 0.5,
  'batch_size': 32,
  'num_gpu': 1,  # 2,

  # Inference
  'max_boxes': 100,
  'iou_threshold': 0.5,
  'score_threshold': 0.5,
}

In [4]:
with open(val_notation) as file:
  data = json.load(file)

easy =[obj for obj in data if int(obj['name'].split('--')[0]) <= 20]
medium = [obj for obj in data if int(obj['name'].split('--')[0]) <= 40 and int(obj['name'].split('--')[0]) > 20]
hard = [obj for obj in data if int(obj['name'].split('--')[0]) > 40]

In [ ]:
model_yolo = MNetYolo(config, class_name, weight_path)

In [ ]:
model_yolo.eval_map(gt_path, predict_path, temp_json, output_ap)

In [ ]:
print(data[3]['name'])
test_image = os.path.join(image_path, data[69]['name'])
model_yolo.predict(test_image)

In [ ]:
model_yolo.export_predict(hard, image_path, predict_path)

In [ ]:
model_yolo.export_gt(hard, image_path, gt_path)

In [ ]:
def decode_notation(img, boxs):
    
    high = img.shape[0]
    wide = img.shape[1]
    
    result = []
    
    for label in boxs:
        x = round(label['data'][0] * wide)
        y = round(label['data'][1] * high)
        w = round(label['data'][2] * wide)
        h = round(label['data'][3] * high)
        result.append({ 'index' : label['index'], 'data' : [x, y, w, h]})
    return result

In [ ]:
def draw_bbox(image, boxs):
    fig, ax = plt.subplots(figsize = (15,15))
    ax.imshow(image, interpolation='nearest')
    boxs = decode_notation(image, boxs)
    print(boxs)
    for box in boxs:
        x1y1 = patches.Circle((int(box['data'][0]), int(box['data'][1])), radius=5, color='blue')
        x2y2 = patches.Circle((int(box['data'][0] + box['data'][2]), int(box['data'][1] + box['data'][3])), radius=5, color='green')
        cpoint = patches.Circle((int(box['data'][0] + (box['data'][2]/2)), int(box['data'][1] + (box['data'][3]/2))), radius=5, color='yellow')
        rect = patches.Rectangle((int(box['data'][0]), int(box['data'][1])), int(box['data'][2]), int(box['data'][3]), linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.add_patch(x1y1)
        ax.add_patch(cpoint)                            
        ax.add_patch(x2y2)

        
    plt.show()

In [ ]:
img = utill.open_image( os.path.join(image_path, data[60]['name']), True)
draw_bbox(img, data[60]['objects'])

In [5]:
def build_model(config, weight_path, load_pretrained=False):
  anchors = np.array(config['anchors']).reshape((2, 7, 2))
  
  inputs = layers.Input(config['image_size'])
  backbone = models.MobileNet(inputs)
  print(backbone.outputs)
  yolo_head = models.FPN_light(backbone, 1, 7)
  basic_model = mdl.Model(inputs, yolo_head)

  if load_pretrained:
    basic_model.load_weights(weight_path)

  model_output = yolo_utils.yolo_detector_light(
            basic_model.outputs, anchors, 1, config['strides'], config['xyscale'])
  nms = utill.nms(model_output, config['image_size'], 1, config['iou_threshold'], config['score_threshold'])
  
  return mdl.Model(basic_model.input, nms)
  

inf_model = build_model(config, weight_path, True)


  

2022-01-07 15:15:42.368278: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-07 15:15:42.368402: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ozmonday): /proc/driver/nvidia/version does not exist
2022-01-07 15:15:42.368951: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[<KerasTensor: shape=(None, 26, 26, 512) dtype=float32 (created by layer 'conv_pw_11_relu')>, <KerasTensor: shape=(None, 13, 13, 1024) dtype=float32 (created by layer 'conv_pw_13_relu')>]


In [6]:
def preprocessing_image(config, img):
  img = img /255
  img = cv2.resize(img, config['image_size'][:2])
  return img


In [7]:
filenames = [line['name'] for line in data[:10]]
img_path = [os.path.join(image_path, line) for line in filenames]
cname = [line.strip() for line in open(class_name).readlines()]

resume = []

for idx in tqdm(range(0, len(img_path), 2)):
  paths = img_path[idx:idx+2]
  imgs = np.zeros((len(paths), *config['image_size']))
  raw_img_shapes = []

  for i, path in enumerate(paths):
    img = utill.open_image(path, False)
    raw_img_shapes.append(img.shape)
    img = preprocessing_image(config, img)
    imgs[i] = img

  b_boxes, b_scores, b_classes, b_valid_detections = inf_model.predict(imgs)
  print(b_boxes)

  for k in range(len(paths)):
    num_boxes = b_valid_detections[k]
    raw_img_shape = raw_img_shapes[k]
    boxes = b_boxes[k, :num_boxes]
    classes = b_classes[k, :num_boxes]
    scores = b_scores[k, :num_boxes]
  
    boxes[:, [0, 2]] = (boxes[:, [0, 2]] * raw_img_shape[1])  # w
    boxes[:, [1, 3]] = (boxes[:, [1, 3]] * raw_img_shape[0])  # h
    cls_names = [cname[int(c)] for c in classes]

    ipath = paths[k]
    filename = ipath.split(os.sep)[-1].split('.')[0]

    objects = []

    output_path = os.path.join(predict_path, filename+'.txt')
    with open(output_path, 'w') as pred_file:
      for box_idx in range(num_boxes):
        b = boxes[box_idx]
        pred_file.write(f'{cls_names[box_idx]} {scores[box_idx]} {b[0]} {b[1]} {b[2]} {b[3]}\n')
        objects.append({"index" : 1+box_idx, "score": scores[box_idx], "data" : [float(b[0]), float(b[1]), float(b[2]), float(b[3])]})
    resume.append({"name": filename, "objects" : objects})






100%|██████████| 5/5 [00:05<00:00,  1.01s/it]


In [ ]:
gt_path = os.path.join('..', 'assets', 'ground_truth')

for img in data[:10]:
  fn = os.path.join(image_path, img['name'])
  im = utill.open_image(fn)
  fn = fn.split(os.sep)[-1].split('.')[0]
  output_path = os.path.join(gt_path, fn+'.txt')
  print(output_path)
  with open(output_path, 'w') as gt_file:
    for object in img['objects']:
      x1 = object['data'][0] * im.shape[0]
      y1 = object['data'][1] * im.shape[1]
      x2 = x1 + (object['data'][2] * im.shape[0])
      y2 = y1 + (object['data'][3] * im.shape[1])
      print('face {} {} {} {}'.format(x1,y1, x2, y2))
      gt_file.write(f'face {x1} {y1} {x2} {y2}\n')




In [ ]:
from glob import glob

gt_counter_per_class = {}
counter_images_per_class = {}
gt_files = []

filename_list = glob(gt_path + '/*.txt')
assert len(filename_list) > 0, 'no ground truth file'
filename_list.sort()

for title in filename_list:
  file_id = title.split(".txt", 1)[0] # ../assets/ground_truth/0_Parade_Parade_0_120
  file_id = os.path.basename(os.path.normpath(file_id)) # 0_Parade_Parade_0_120
  temp_path = os.path.join(predict_path, (file_id + ".txt"))
  assert os.path.exists(temp_path), "Error. File not found: {}\n".format(temp_path)
  lines_list = utill.read_txt_to_list(title)
  
  bounding_boxes = []
  already_seen_classes = []
  for line in lines_list:
    class_name, left, top, right, bottom = line.split()
    bbox = left + " " + top + " " + right + " " + bottom
    bounding_boxes.append({"class_name": class_name, "bbox": bbox, "used": False})
    
    # count that object
    if class_name in gt_counter_per_class:
      gt_counter_per_class[class_name] += 1
    else:
      # if class didn't exist yet
      gt_counter_per_class[class_name] = 1

    if class_name not in already_seen_classes:
      if class_name in counter_images_per_class:
        counter_images_per_class[class_name] += 1
      else:
        # if class didn't exist yet
        counter_images_per_class[class_name] = 1
      already_seen_classes.append(class_name)

  new_temp_file = os.path.join(temp_json, file_id+"_ground_truth.json") #TEMP_FILES_PATH + "/" + file_id + "_ground_truth.json"
  gt_files.append(new_temp_file)
  with open(new_temp_file, 'w') as outfile:
    json.dump(bounding_boxes, outfile)

gt_classes = list(gt_counter_per_class.keys())
# let's sort the classes alphabetically
gt_classes = sorted(gt_classes)
n_classes = len(gt_classes)
print(gt_classes, gt_counter_per_class)

dr_files_list = sorted(glob(os.path.join(predict_path, '*.txt')))
# print(dr_files_list)

for class_index, class_name in enumerate(gt_classes):
  bounding_boxes = []
  for txt_file in dr_files_list:
    file_id = txt_file.split(".txt", 1)[0]
    file_id = os.path.basename(os.path.normpath(file_id))
    temp_path = os.path.join(gt_path, (file_id + ".txt"))
    if class_index == 0 :
      if not os.path.exists(temp_path):
        error_msg = f"Error. File not found: {temp_path}\n"
        print(error_msg)

    lines = utill.read_txt_to_list(txt_file)
    for line in lines:
      try:
        tmp_class_name, confidence, left, top, right, bottom = line.split()
      except ValueError:
        error_msg = f"""Error: File {txt_file} in the wrong format.\n 
                        Expected: <class_name> <confidence> <left> <top> <right> <bottom>\n 
                        Received: {line} \n"""
        print(error_msg)
      if tmp_class_name == class_name:
        bbox = left + " " + top + " " + right + " " + bottom
        bounding_boxes.append({"confidence": confidence, "file_id": file_id, "bbox": bbox})

  bounding_boxes.sort(key=lambda x: float(x['confidence']), reverse=True)

  with open(temp_json + "/" + class_name + "_dr.json", 'w') as outfile:
    json.dump(bounding_boxes, outfile)

sum_AP = 0.0
ap_dictionary = {}

with open(output_ap + "/output.txt", 'w') as output_file:
  output_file.write("# AP and precision/recall per class\n")
  count_true_positives = {}
  
  for class_index, class_name in enumerate(gt_classes):

    count_true_positives[class_name] = 0

    dr_file = os.path.join(temp_json, class_name + "_dr.json")
    # print(dr_file)
    dr_data = json.load(open(dr_file))
    # print(len(dr_data))

    nd = len(dr_data)
    tp = [0] * nd
    # print(len(tp))
    fp = [0] * nd
  
    for idx, detection in enumerate(dr_data):
      file_id = detection["file_id"]
      # print(f'filename : {file_id}')
      gt_file = os.path.join(temp_json, file_id + "_ground_truth.json")
      # print(gt_file)
      ground_truth_data = json.load(open(gt_file))
      # print(len(ground_truth_data))
      ovmax = -1
      gt_match = -1

      bb = [float(x) for x in detection["bbox"].split()]
      # print(f'bonding_box prediction: {bb}')

      for obj in ground_truth_data:
        if obj["class_name"] == class_name:
          bbgt = [float(x) for x in obj["bbox"].split()]
          bi = [max(bb[0], bbgt[0]), max(bb[1], bbgt[1]), min(bb[2], bbgt[2]), min(bb[3], bbgt[3])]
          # print(f'bi : {bi}')
          iw = bi[2] - bi[0] 
          ih = bi[3] - bi[1]
          # print(iw)
          # print(ih)

          if iw > 0 and ih > 0:
            # compute overlap (IoU) = area of intersection / area of union
            ua = ((bb[2] - bb[0]) * (bb[3] - bb[1])) + ((bbgt[2] - bbgt[0]) * (bbgt[3] - bbgt[1])) - (iw * ih)
            ov = (iw * ih) / ua
            # print(f'ov : {ov}')
            if ov > ovmax:
              ovmax = ov
              gt_match = obj
      
      # print(f'gt_match : {gt_match}')
      # print(f'ovmax : {ovmax} \n')
      
      min_overlap = 0.05
      if ovmax >= min_overlap:
        if not bool(gt_match["used"]):
          # true positive
          tp[idx] = 1
          gt_match["used"] = True
          count_true_positives[class_name] += 1
          # update the ".json" file
          with open(gt_file, 'w') as f:
            f.write(json.dumps(ground_truth_data))
        else:
          # false positive (multiple detection)
          fp[idx] = 1
      else:
        fp[idx] = 1

    cumsum = 0
    for idx, val in enumerate(fp):
      fp[idx] += cumsum
      cumsum += val
    print('fp ', cumsum)
    cumsum = 0
    for idx, val in enumerate(tp):
      tp[idx] += cumsum
      cumsum += val
    print('tp ', cumsum)
    rec = tp[:]
    for idx, val in enumerate(tp):
      rec[idx] = float(tp[idx]) / gt_counter_per_class[class_name]
    print('recall ', cumsum)
    prec = tp[:]
    for idx, val in enumerate(tp):
      prec[idx] = float(tp[idx]) / (fp[idx] + tp[idx])
    print('prec ', cumsum)

 


    # print(temp_path)
    # print(file_id)

#   print(already_seen_classes)
# print(gt_counter_per_class)
# print(counter_images_per_class)
    


  